In [10]:
import __future__
import codecs, os, re
from bs4 import BeautifulSoup

In [11]:
baseurl='https://cmsweb.cern.ch/dqm/offline/data/browse/ROOT/OfflineData/'
url_runs_2018="Run2018/ZeroBias/"
url_runs_2017="Run2017/ZeroBias/"
url_runs_2016="Run2016/ZeroBias/"
# url_runs_2015="Run2015/ZeroBias/"


url_runs=[url_runs_2018,url_runs_2017,url_runs_2016]#,url_runs_2015]
print "Fetching from: "+baseurl
print "list of relative urls: ",url_runs

Fetching from: https://cmsweb.cern.ch/dqm/offline/data/browse/ROOT/OfflineData/
list of relative urls:  ['Run2018/ZeroBias/', 'Run2017/ZeroBias/', 'Run2016/ZeroBias/']


# This section shows the procedure to parse one instance and should be adapted in a for loop for all runs and all years

We first create dump the html page into a document using `curl` command and specifying the path to the certificate.
In this example we are looking at runs from 2017

In [18]:
os.popen("curl -k --cert proxy.cert --key proxy.cert -X GET "+baseurl+url_runs_2018+" > index.html") 
# This dumps the html page into a file which I called "index.html" 
#Check the directory you are working in to see if it's there

f=codecs.open("index.html",'rb')       # open the document
index = f.readlines()                  # read the document
a=str(index)                           # convert to string
soup = BeautifulSoup(a, 'html.parser') # create the soup object

## Just to see if it works

In [19]:
soup

[]

In [14]:
print soup.prettify()

[]



In [13]:
print soup.title.contents

AttributeError: 'NoneType' object has no attribute 'contents'

In [6]:
print soup.a # This will show the first instance of and anchor

<a href="/dqm/offline/data/browse/ROOT/OfflineData/Run2017">Up</a>


## One way to parse the page

In [5]:
# We want to get all the anchor tags <a>...<\a> from the body of the HTML page 
tag = soup.body.find_all("a")

print "tag 10 is "+tag[10].string

RUNSXX=[]
i=1
for child in tag:
    if child.string != 'Up':
        print "%s)"%i,child.string  
        i+=1
        RUNSXX.append(child.string)
        
print "RUNSXX has this many items:",len(RUNSXX)

AttributeError: 'NoneType' object has no attribute 'find_all'

## Another way of finding the runs using Regular Expressions

Since we know that the list of runs all share the first `000` we can use Regular expressions to create a list much like before with one single line

In [9]:
RUNSXXRE= soup.body.find_all(string=re.compile("000.")) 
# the "." for a regular expresions means that it will expect ANY character EXCEPT newlines

# print len(RUNSXXRE),"items"
print RUNSXXRE
if len(RUNSXXRE)==0:
    print "List is empty"
else:
    print "List has",len(RUNSXXRE),"items"

AttributeError: 'NoneType' object has no attribute 'find_all'

 ## Time to parse and collect all run numbers for one specific year 

In [12]:
os.popen("curl -k --cert proxy.cert --key proxy.cert -X GET "+baseurl+url_runs_2018+" > index.html") 
# This dumps the html page into a file which I called "index.html" 
#Check the directory you are working in to see if it's there

f=codecs.open("index.html",'rb')       # open the document
index = f.readlines()                  # read the document
a=str(index)                           # convert to string
soup = BeautifulSoup(a, 'html.parser') # create the soup object
RUNSXXRE= soup.body.find_all(string=re.compile("000.")) 

rundir_with_rootfiles=[]
runs_with_rootfiles=[]
rundir_without_rootfiles=[]
# runs_without_rootfiles=[]


for rundir in RUNSXXRE:
    os.popen("curl -k --cert proxy.cert --key proxy.cert -X GET "+baseurl+url_runs_2018+rundir+" > index.html") 
    f=codecs.open("index.html",'rb')       # open the document
    index = f.readlines()                  # read the document
    a=str(index)                           # convert to string
    soup = BeautifulSoup(a, 'html.parser') # create the soup object
    entries= soup.body.find_all(string=re.compile("DQM."))
    if len(entries)==0:
        rundir_without_rootfiles.append(str(rundir)) # Fill the list
        #print runs,"is empty"
    else:
        
        for n in re.findall(r"R(\d+)",str(entries)): # This will only keep the 6 digits of the run numbers that we need (without the "R000")
            x=str(int(n))                            # Since we wont want to do math with these numbers I will convert them to strings
            runs_with_rootfiles.append(x)            # Fill the list
        rundir_with_rootfiles.append(str(rundir))         # Fill the list
        
        #print runs,"has",len(entries),"root files"    

AttributeError: 'NoneType' object has no attribute 'find_all'

In [8]:
print "==========================================================="
print "For",url_runs_2018
print "Out of a total of",len(RUNSXXRE),"run directories:\n",len(rundir_with_rootfiles),"have root files\n",len(rundir_without_rootfiles),"are empty"
print "There are",len(runs_with_rootfiles),"runs with root files"
# print "\n"
# print "List of available runs for this year\n\n",runs_with_rootfiles

For Run2018/ZeroBias/
Out of a total of

NameError: name 'RUNSXXRE' is not defined

# Here is for all runs and all years

In [3]:
baseurl='https://cmsweb.cern.ch/dqm/offline/data/browse/ROOT/OfflineData/'
url_runs_2018="Run2018/ZeroBias/"
url_runs_2017="Run2017/ZeroBias/"
url_runs_2016="Run2016/ZeroBias/"
# url_runs_2015="Run2015/ZeroBias/"


url_runs=[url_runs_2018,url_runs_2017,url_runs_2016]#,url_runs_2015]
print "Fetching from: "+baseurl
print "list of relative urls: ",url_runs

global_rundir_with_rootfiles=[]
global_runs_with_rootfiles=[]
global_rundir_without_rootfiles=[]

# runs_without_rootfiles=[]
for url in url_runs:
    os.popen("curl -k --cert proxy.cert --key proxy.cert -X GET "+baseurl+url+" > index.html") 
    f=codecs.open("index.html",'rb')       # open the document
    index = f.readlines()                  # read the document
    a=str(index)                           # convert to string
    soup = BeautifulSoup(a, 'html.parser') # create the soup object
    RUNSXXRE= soup.body.find_all(string=re.compile("000.")) # the "." for a regular expresions means that it will expect ANY character EXCEPT newlines
  
    rundir_with_rootfiles=[]
    runs_with_rootfiles=[]
    rundir_without_rootfiles=[]
    
    
    for rundir in RUNSXXRE:
        os.popen("curl -k --cert proxy.cert --key proxy.cert -X GET "+baseurl+url+rundir+" > index.html") 
        f=codecs.open("index.html",'rb')       # open the document
        index = f.readlines()                  # read the document
        a=str(index)                           # convert to string
        soup = BeautifulSoup(a, 'html.parser') # create the soup object
        entries= soup.body.find_all(string=re.compile("DQM."))   
        if len(entries)==0:
            rundir_without_rootfiles.append(str(rundir))        # Fill the list
            global_rundir_without_rootfiles.append(str(rundir)) # Fill the list
            #print rundir,"is empty"
        else:

            for n in re.findall(r"R(\d+)",str(entries)):    # This will only keep the 6 digits of the run numbers that we need (without the "R000")
                x=str(int(n))                               # Since we wont want to do math with these numbers I will convert them to strings
                runs_with_rootfiles.append(x)               # Fill the list
                global_runs_with_rootfiles.append(x)        # Fill the list
                
            rundir_with_rootfiles.append(str(rundir))         # Fill the list
            global_rundir_with_rootfiles.append(str(rundir))  # Fill the list

            #print rundir,"has",len(entries),"root files"    
    print "==========================================================="
    print "For",url
    print "Out of a total of",len(RUNSXXRE),"run directories:\n",len(rundir_with_rootfiles),"have root files\n",len(rundir_without_rootfiles),"are empty"
    print "There are",len(runs_with_rootfiles),"runs with root files"
    print "\n"
    #print "List of available runs for this year\n\n",runs_with_rootfiles
    
print "==========================================================="
print "For GLOBAL"
print "Out of all run directories:\n",len(global_rundir_with_rootfiles),"have root files\n",len(global_rundir_without_rootfiles),"are empty"
print "There are",len(global_runs_with_rootfiles),"runs with root files"
print "\n"


Fetching from: https://cmsweb.cern.ch/dqm/offline/data/browse/ROOT/OfflineData/
list of relative urls:  ['Run2018/ZeroBias/', 'Run2017/ZeroBias/', 'Run2016/ZeroBias/']
For Run2018/ZeroBias/
Out of a total of 106 run directories:
73 have root files
33 are empty
There are 1185 runs with root files


For Run2017/ZeroBias/
Out of a total of 102 run directories:
27 have root files
75 are empty
There are 286 runs with root files


For Run2016/ZeroBias/
Out of a total of 104 run directories:
0 have root files
104 are empty
There are 0 runs with root files


For GLOBAL
Out of all run directories:
100 have root files
212 are empty
There are 1471 runs with root files




### If you want to see the diferent list of runs just do 

In [10]:
# For runs WITH root files
global_rundir_with_rootfiles[1]

'0003256xx/'

### Catching the repeated instances of the run numbers due to variations of the root file names

In [11]:
i=0
repeatruns=[]
for run in global_runs_with_rootfiles: 
    if run == global_runs_with_rootfiles[-1]:
#         print "Last run",run
        break
    if run==global_runs_with_rootfiles[i+1]:
        #print "repeated run:",run
        repeatruns.append(run)
    i+=1
#print 'there are',len(repeatruns),'repeated runs'

In [12]:
repeatruns

['319910',
 '318877',
 '317182',
 '316218',
 '306459',
 '306459',
 '306459',
 '306459',
 '306459',
 '306459',
 '306456',
 '306456',
 '306456',
 '306456',
 '306456',
 '306432',
 '306422',
 '306417',
 '306171',
 '306170',
 '306169',
 '306155',
 '306154',
 '306139',
 '306137',
 '306135',
 '306126',
 '306125',
 '306125',
 '306095',
 '306093',
 '306091',
 '306051',
 '306042',
 '306041',
 '305902',
 '305842',
 '305841',
 '305832',
 '305821',
 '305636',
 '305590',
 '305588',
 '305586',
 '305518',
 '305517',
 '305406',
 '305365',
 '305350',
 '305341',
 '305338',
 '305312',
 '305311',
 '305310',
 '305252',
 '305248',
 '305247',
 '305237',
 '305236',
 '305202',
 '305188',
 '305186',
 '305185',
 '305184',
 '305182',
 '305181',
 '305180',
 '305178',
 '305114',
 '305112',
 '305081',
 '305064',
 '305064',
 '305064',
 '305064',
 '305064',
 '305064',
 '305062',
 '305059',
 '305046',
 '305040',
 '299481',
 '299481',
 '299481',
 '299480',
 '299480',
 '299480',
 '299479',
 '299479',
 '299479',
 '299478',

In [13]:
# For run directories without root files
global_rundir_without_rootfiles

['0003218xx/',
 '0003217xx/',
 '0003216xx/',
 '0003215xx/',
 '0003214xx/',
 '0003213xx/',
 '0003212xx/',
 '0003211xx/',
 '0003210xx/',
 '0003209xx/',
 '0003208xx/',
 '0003207xx/',
 '0003206xx/',
 '0003205xx/',
 '0003204xx/',
 '0003203xx/',
 '0003202xx/',
 '0003201xx/',
 '0003192xx/',
 '0003191xx/',
 '0003186xx/',
 '0003185xx/',
 '0003184xx/',
 '0003183xx/',
 '0003182xx/',
 '0003181xx/',
 '0003180xx/',
 '0003179xx/',
 '0003178xx/',
 '0003177xx/',
 '0003074xx/',
 '0003073xx/',
 '0003072xx/',
 '0003071xx/',
 '0003070xx/',
 '0003069xx/',
 '0003068xx/',
 '0003067xx/',
 '0003066xx/',
 '0003065xx/',
 '0003063xx/',
 '0003062xx/',
 '0003048xx/',
 '0003047xx/',
 '0003046xx/',
 '0003045xx/',
 '0003044xx/',
 '0003043xx/',
 '0003042xx/',
 '0003041xx/',
 '0003040xx/',
 '0003039xx/',
 '0003038xx/',
 '0003037xx/',
 '0003035xx/',
 '0003029xx/',
 '0003028xx/',
 '0003026xx/',
 '0003025xx/',
 '0003024xx/',
 '0003023xx/',
 '0003022xx/',
 '0003021xx/',
 '0003020xx/',
 '0003019xx/',
 '0003018xx/',
 '0003016x

# Lets use the JSON files to find the runs that we dont have

In [13]:
baseurl='https://cms-service-dqm.web.cern.ch/cms-service-dqm/CAF/certification/'
json_runs_2018="Collisions18/13TeV/ReReco/Cert_314472-325175_13TeV_17SeptEarlyReReco2018ABC_PromptEraD_Collisions18_JSON.txt"
json_runs_2017="Collisions17/13TeV/ReReco/Cert_294927-306462_13TeV_EOY2017ReReco_Collisions17_JSON_v1.txt"
json_runs_2016="Collisions16/13TeV/ReReco/Final/Cert_271036-284044_13TeV_ReReco_07Aug2017_Collisions16_JSON.txt"
# json_runs_2015="Run2015/ZeroBias/"


json_runs=[json_runs_2018,json_runs_2017,json_runs_2016]#,url_runs_2015]
print "Fetching from: "+baseurl
print "list of relative urls: ",json_runs

Fetching from: https://cms-service-dqm.web.cern.ch/cms-service-dqm/CAF/certification/
list of relative urls:  ['Collisions18/13TeV/ReReco/Cert_314472-325175_13TeV_17SeptEarlyReReco2018ABC_PromptEraD_Collisions18_JSON.txt', 'Collisions17/13TeV/ReReco/Cert_294927-306462_13TeV_EOY2017ReReco_Collisions17_JSON_v1.txt', 'Collisions16/13TeV/ReReco/Final/Cert_271036-284044_13TeV_ReReco_07Aug2017_Collisions16_JSON.txt']


In [14]:
json_runslist=[]
for url in json_runs:
    os.popen("curl -k -X GET "+baseurl+url+" > index.html") 



    # os.popen("curl -k --cert proxy.cert --key proxy.cert -X GET "+baseurl+url_runs_2017+" > index.html") 
    # This dumps the html page into a file which I called "index.html" 
    #Check the directory you are working in to see if it's there

    f=codecs.open("index.html",'rb')       # open the document
    index = f.readlines()                  # read the document
    a=str(index)                           # convert to string
    soup = BeautifulSoup(a, 'html.parser') # create the soup object

    a=re.findall(r"(\d{6})",str(soup)) #find exactly 6 occurrences of a number
                                    #in other words find 6 digit numbers

    print "amount of runs in the json",len(a),"in",url
    print a[0],"to",a[-1]
    json_runslist.append(a)
    


amount of runs in the json 478 in Collisions18/13TeV/ReReco/Cert_314472-325175_13TeV_17SeptEarlyReReco2018ABC_PromptEraD_Collisions18_JSON.txt
315257 to 325172
amount of runs in the json 474 in Collisions17/13TeV/ReReco/Cert_294927-306462_13TeV_EOY2017ReReco_Collisions17_JSON_v1.txt
297050 to 306460
amount of runs in the json 393 in Collisions16/13TeV/ReReco/Final/Cert_271036-284044_13TeV_ReReco_07Aug2017_Collisions16_JSON.txt
273158 to 284044


## Finding all missing runs for all years

In [15]:
json_list = [item for sublist in json_runslist for item in sublist]
len(json_list)

1345

In [41]:
good=[]
missing=[]
bad=[]
for jrun in json_list:
    for grun in global_runs_with_rootfiles:
        if jrun==grun:
            good.append(jrun)            
    if jrun not in global_runs_with_rootfiles:
        missing.append(jrun)
        
for grun in global_runs_with_rootfiles:
    if grun not in json_list:
        bad.append(grun)
# print "all good runs found",good,"\n"
print len(good),"good runs"
print len(missing),"missing runs"
print len(bad),"bad runs"



580 good runs
859 missing runs
891 bad runs
